# SageMaker ML Algorithm 유형

- Built-in Algorithm
- Script Mode
- Custom Image
- AWS Market

In [ ]:
import sagemaker

role = 'arn:aws:iam::834160605896:role/service-role/AmazonSageMaker-ExecutionRole-20210317T125074'

## Built-in Algorithm

- Sagemaker에서 제공하는 training 알고리즘으로, 사용자의 문제 유형에 적합한 알고리즘을 선택하여 학습
- 사용자가 모델 네트워크를 구성할 필요 없이 최소한의 노력으로 빠르게 학습이 가능

아래는 집의 특성 data를 기반으로 집값을 예측하는 회귀 모델 학습입니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("kc_house_data.csv")

# 훈련, 검증 및 검정 분할
ys = np.array(df['price']).astype("float32")
xs = np.array(df['sqft_living']).astype("float32").reshape(-1,1)

np.random.seed(8675309)
train_features, test_features, train_labels, test_labels = train_test_split(xs, ys, test_size=0.2)
val_features, test_features, val_labels, test_labels = train_test_split(test_features, test_labels, test_size=0.5)

In [ ]:
# 모델 정의
linear_model = sagemaker.LinearLearner(role=role,
                                       base_job_name='0521-algo-builtin',
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       predictor_type='regressor')

In [ ]:
# 학습
train_records = linear_model.record_set(train_features, train_labels, channel='train')
val_records = linear_model.record_set(val_features, val_labels, channel='validation')
test_records = linear_model.record_set(test_features, test_labels, channel='test')

linear_model.fit([train_records, val_records, test_records], logs=False)

In [ ]:
# 결과 확인
sagemaker.analytics.TrainingJobAnalytics(linear_model._current_job_name, metric_names = ['test:mse', 'test:absolute_loss']).dataframe()


---

## Script Mode

- Scikit-learn, Spark ML, XGBoost, Tensorflow, Keras, Pytorch, MXNet, Chainer 지원
- 사용할 framework 버전은 사용자가 지정할 수 있으며, compatible한 cuda, cudnn이 자동으로 설치되는 것으로 보임
- 3rd-party package로 PyPi만 사용이 가능
- 사용자는 training script를 작성하여 해당 환경에서 학습을 수행할 수 있음
- Script Mode를 통한 training 수행 시 job 정보가 환경변수로 제공됨 (이를 활용하여 training script를 작성하도록 가이드되어있음) ([tensorflow 예시 링크](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html))

아래는 `train_aws.py` 코드를 pytorch 1.7.1 환경에서 학습합니다.  
cifar10 dataset에 대한 pytorch resnet 학습 코드이며,  
hyperparameter로 batch, lr, epochs를 받습니다.

In [ ]:
from sagemaker.pytorch.estimator import PyTorch

pytorch_estimator = PyTorch('train_scriptmode.py',
                            framework_version='1.7.1',
                            py_version='py3',
                            role=role,
                            base_job_name='0521-algo-scriptmode',
                            instance_count=1,
                            instance_type='ml.p3.2xlarge',
                            hyperparameters = {'epochs': 1},
                            metric_definitions=[
                               {'Name': 'accuracy', 'Regex': 'Test Accuracy: (.*?);'}
                            ])

In [ ]:
pytorch_estimator.fit()

---

## Custom Image

- 사용자가 학습에 필요한 환경을 직접 Dockerfile로 구성하고 빌드하여 ECR에 등록할 수 있음
- container 실행 시 training 수행이 가능하도록 Dockerfile 내에서 training을 수행하기 위한 명령을 함께 빌드해야 함
- ECR 주소만 안다면 외부와도 공유가 가능
- 외부에 공유되고 있는 Image를 Sagemaker에 맞게 한번 더 빌드하여 사용할 수 있음 (또는 AWS Market에 공유된 이미지 사용)

위와 동일한 cifar10 dataset에 대한 pytorch resnet을 학습합니다.  
아래 Dockerfile을 사용하였습니다.
```Dockerfile
FROM nexus-registry.sel-d4.sr-cloud.com:12000/ubuntu16.04-python3.7-cuda10.1-cudnn7.6-pytorch1.0.1:latest
RUN pip3 install sagemaker-training
RUN pip3 install Pillow==6.1
COPY train_aws.py /opt/ml/code/train.py
ENV SAGEMAKER_PROGRAM train.py
```

In [ ]:
from sagemaker.estimator import Estimator

estimator = Estimator(image_uri='834160605896.dkr.ecr.ap-northeast-2.amazonaws.com/mlp/mlp-base-image:pytorch-2',
                      role=role,
                      base_job_name='0521-algo-customimage',
                      instance_count=1,
                      instance_type='ml.p2.xlarge')

# start training
estimator.fit()